In [13]:
import tensorflow as tf
import numpy as np

# 1. 데이터 정의 (여기서는 학습 데이터를 임의로 정의합니다. 실제로는 데이터를 제공해야 합니다.)
x = np.linspace(0, 1, 100)
y = np.linspace(0, 1, 100)
t = np.linspace(0, 1, 100)

X, Y, T = np.meshgrid(x, y, t)

# 학습 데이터 구조 변경
train_data = np.vstack((X.flatten(), Y.flatten(), T.flatten())).T
train_data = tf.convert_to_tensor(train_data, dtype=tf.float32)

# 2. 신경망 정의
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(3,)),  # x, y, t를 입력으로 받습니다.
    tf.keras.layers.Dense(20, activation='LeakyReLU'),
    tf.keras.layers.Dense(20, activation='LeakyReLU'),
    tf.keras.layers.Dense(1)  # u의 값을 출력합니다.
])

def pde_loss(model, x):
    with tf.GradientTape(persistent=True) as outer_tape:
        outer_tape.watch(x)
        with tf.GradientTape(persistent=True) as inner_tape:
            inner_tape.watch(x)
            u = model(x)
        
        u_t = outer_tape.gradient(u, x)[:, 0]  # u를 t에 대해서 미분
        u_x = inner_tape.gradient(u, x)[:, 1]  # u를 x에 대해서 미분
        u_y = inner_tape.gradient(u, x)[:, 2]  # u를 y에 대해서 미분
    
    with tf.GradientTape(persistent=True) as second_tape:
        second_tape.watch(x)
        u_x = second_tape.gradient(u, x)[:, 1]
        u_y = second_tape.gradient(u, x)[:, 2]

    u_xx = second_tape.gradient(u_x, x)[:, 1]
    u_yy = second_tape.gradient(u_y, x)[:, 2]

    del inner_tape
    del outer_tape
    del second_tape

    D = 0.1  # 예를 들어 확산 계수를 0.1로 가정
    f = u_t - D * (u_xx + u_yy)  # 미분 방정식의 차이

    return tf.reduce_mean(tf.square(f))

# 4. 학습
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# 학습 과정 정의
def train_step():
    with tf.GradientTape() as tape:
        loss = pde_loss(model, train_data)
    
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    
    return loss

In [14]:
# 학습
epochs = 5000
for epoch in range(epochs):
    loss = train_step()
    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

# 이렇게 학습을 진행한 후에는 model을 사용하여 (x, y, t) 위치에서의 u 값을 예측할 수 있습니다.

TypeError: 'NoneType' object is not subscriptable